In [2]:
import tensorflow as tf
import numpy as np

In [3]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [5]:
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [6]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [7]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

In [8]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[None, n_step, n_input])
Y = tf.placeholder(dtype=tf.int32, shape=[None])

In [9]:
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [10]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [11]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [25]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

In [26]:
model = tf.matmul(outputs, W) + b

In [27]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

In [28]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [29]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 0001 cost = 2.847366
Epoch: 0002 cost = 2.058687
Epoch: 0003 cost = 1.441775
Epoch: 0004 cost = 0.854716
Epoch: 0005 cost = 0.878930
Epoch: 0006 cost = 0.515260
Epoch: 0007 cost = 0.428536
Epoch: 0008 cost = 0.205685
Epoch: 0009 cost = 0.261461
Epoch: 0010 cost = 0.139727
Epoch: 0011 cost = 0.236743
Epoch: 0012 cost = 0.161559
Epoch: 0013 cost = 0.381628
Epoch: 0014 cost = 0.098940
Epoch: 0015 cost = 0.131993
Epoch: 0016 cost = 0.067314
Epoch: 0017 cost = 0.013192
Epoch: 0018 cost = 0.046398
Epoch: 0019 cost = 0.180739
Epoch: 0020 cost = 0.036034
Epoch: 0021 cost = 0.028433
Epoch: 0022 cost = 0.228847
Epoch: 0023 cost = 0.263208
Epoch: 0024 cost = 0.319425
Epoch: 0025 cost = 0.406820
Epoch: 0026 cost = 0.152425
Epoch: 0027 cost = 0.038515
Epoch: 0028 cost = 0.005831
Epoch: 0029 cost = 0.006634
Epoch: 0030 cost = 0.032614
optimization finished!


In [30]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('input value:', [w[:3] + ' ' for w in seq_data])
print('predicted value:', predict_words)
print('accuracy:', accuracy_val)

input value: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
predicted value: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
accuracy: 1.0


In [32]:
predict

array([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])